In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention

2024-05-13 19:30:26.130157: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 19:30:26.130335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 19:30:26.283768: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
titles_df = pd.read_csv('/kaggle/input/imdb-compressed/title.basics.tsv', sep = '\t')

/tmp/ipykernel_502/3074310362.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_df = pd.read_csv('/kaggle/input/imdb-compressed/title.basics.tsv', sep = '\t')


In [3]:
print(titles_df)

             tconst  titleType               primaryTitle  \
0         tt0000001      short                 Carmencita   
1         tt0000002      short     Le clown et ses chiens   
2         tt0000003      short             Pauvre Pierrot   
3         tt0000004      short                Un bon bock   
4         tt0000005      short           Blacksmith Scene   
...             ...        ...                        ...   
10756655  tt9916848  tvEpisode              Episode #3.17   
10756656  tt9916850  tvEpisode              Episode #3.19   
10756657  tt9916852  tvEpisode              Episode #3.20   
10756658  tt9916856      short                   The Wind   
10756659  tt9916880  tvEpisode  Horrid Henry Knows It All   

                      originalTitle isAdult startYear endYear runtimeMinutes  \
0                        Carmencita       0      1894      \N              1   
1            Le clown et ses chiens       0      1892      \N              5   
2                    Pauvre

In [4]:
# Load the IMDb ratings dataset
ratings_df = pd.read_csv('/kaggle/input/imdb-compressed/title.ratings.tsv', sep='\t')

In [5]:
print(ratings_df)

            tconst  averageRating  numVotes
0        tt0000001            5.7      2052
1        tt0000002            5.7       274
2        tt0000003            6.5      2006
3        tt0000004            5.4       179
4        tt0000005            6.2      2773
...            ...            ...       ...
1435451  tt9916730            7.0        12
1435452  tt9916766            7.1        23
1435453  tt9916778            7.2        36
1435454  tt9916840            7.0         9
1435455  tt9916880            8.5         7

[1435456 rows x 3 columns]


In [6]:
basics_df = pd.read_csv('/kaggle/input/imdb-compressed/name.basics.tsv', sep='\t')

In [7]:
print(basics_df)

             nconst         primaryName birthYear deathYear  \
0         nm0000001        Fred Astaire      1899      1987   
1         nm0000002       Lauren Bacall      1924      2014   
2         nm0000003     Brigitte Bardot      1934        \N   
3         nm0000004        John Belushi      1949      1982   
4         nm0000005      Ingmar Bergman      1918      2007   
...             ...                 ...       ...       ...   
13473413  nm9993714   Romeo del Rosario        \N        \N   
13473414  nm9993716       Essias Loberg        \N        \N   
13473415  nm9993717  Harikrishnan Rajan        \N        \N   
13473416  nm9993718         Aayush Nair        \N        \N   
13473417  nm9993719          Andre Hill        \N        \N   

                            primaryProfession  \
0                actor,miscellaneous,producer   
1          actress,soundtrack,archive_footage   
2           actress,music_department,producer   
3               actor,writer,music_department  

In [8]:
crews_df = pd.read_csv('/kaggle/input/imdb-compressed/title.crew.tsv', sep='\t')

In [9]:
print(crews_df)

             tconst            directors  \
0         tt0000001            nm0005690   
1         tt0000002            nm0721526   
2         tt0000003            nm0721526   
3         tt0000004            nm0721526   
4         tt0000005            nm0005690   
...             ...                  ...   
10116731  tt9916848            nm1485677   
10116732  tt9916850            nm1485677   
10116733  tt9916852            nm1485677   
10116734  tt9916856           nm10538645   
10116735  tt9916880  nm0584014,nm0996406   

                                                    writers  
0                                                        \N  
1                                                        \N  
2                                                        \N  
3                                                        \N  
4                                                        \N  
...                                                     ...  
10116731  nm9187127,nm1485677,nm98263

In [10]:
# Merge ratings_df, crews_df, and basics_df on 'tconst'
merged_df_2 = pd.merge(ratings_df, crews_df, on='tconst', how='inner')
merged_df_2 = pd.merge(merged_df_2, titles_df, on='tconst', how='inner' )
merged_df_2 = merged_df_2.drop(['originalTitle','isAdult','endYear','runtimeMinutes','numVotes'], axis=1)
print(merged_df_2)

            tconst  averageRating            directors              writers  \
0        tt0000001            5.7            nm0005690                   \N   
1        tt0000002            5.7            nm0721526                   \N   
2        tt0000003            6.5            nm0721526                   \N   
3        tt0000004            5.4            nm0721526                   \N   
4        tt0000005            6.2            nm0005690                   \N   
...            ...            ...                  ...                  ...   
1422489  tt9916730            7.0           nm10538612           nm10538612   
1422490  tt9916766            7.1                   \N                   \N   
1422491  tt9916778            7.2                   \N                   \N   
1422492  tt9916840            7.0  nm0584014,nm0996406  nm1482639,nm2586970   
1422493  tt9916880            8.5  nm0584014,nm0996406  nm1482639,nm2586970   

         titleType                primaryTitle star

In [11]:
# merged_df_2 = pd.merge(merged_df_2, basics_df, on='tconst', how='inner')

In [12]:
# Keep rows where titleType is 'movie'
merged_df_2 = merged_df_2[merged_df_2['titleType'] == 'movie']

# Print the filtered DataFrame
print(merged_df_2)

            tconst  averageRating   directors                        writers  \
8        tt0000009            5.4   nm0085156                      nm0085156   
143      tt0000147            5.2   nm0714557                             \N   
336      tt0000502            4.4   nm0063413  nm0063413,nm0657268,nm0675388   
370      tt0000574            6.0   nm0846879                      nm0846879   
378      tt0000591            5.4   nm0141150                      nm0141150   
...            ...            ...         ...                            ...   
1422463  tt9916362            6.4   nm1893148            nm1893148,nm3471432   
1422468  tt9916428            3.6   nm0910951                             \N   
1422474  tt9916538            8.6   nm4457074  nm4843252,nm4900525,nm2679404   
1422487  tt9916706            7.6   nm7764440                      nm7933903   
1422489  tt9916730            7.0  nm10538612                     nm10538612   

        titleType                   pri

In [13]:
# Split the 'writers' column by commas and expand it into separate columns
writers_columns = merged_df_2['writers'].str.split(',', expand=True)

# Rename the columns to make them more meaningful
writers_columns.columns = [f"Writer_{i+1}" for i in range(writers_columns.shape[1])]

# Concatenate the new columns with the original DataFrame
merged_df_2 = pd.concat([merged_df_2, writers_columns], axis=1)

# Drop the original 'writers' column
merged_df_2.drop('writers', axis=1, inplace=True)

In [14]:
# Split the 'directors' column by commas and expand it into separate columns
directors_columns = merged_df_2['directors'].str.split(',', expand=True)

# Rename the columns to make them more meaningful
directors_columns.columns = [f"Director_{i+1}" for i in range(directors_columns.shape[1])]

# Concatenate the new columns with the original DataFrame
merged_df_2 = pd.concat([merged_df_2, directors_columns], axis=1)

# Drop the original 'directors' column
merged_df_2.drop('directors', axis=1, inplace=True)

In [15]:
# Split the 'genres' column by commas and expand it into separate columns
genres_columns = merged_df_2['genres'].str.split(',', expand=True)

# Rename the columns to make them more meaningful
genres_columns.columns = [f"Genre_{i+1}" for i in range(genres_columns.shape[1])]

# Concatenate the new columns with the original DataFrame
merged_df_2 = pd.concat([merged_df_2, genres_columns], axis=1)

# Drop the original 'genres' column
merged_df_2.drop('genres', axis=1, inplace=True)

In [22]:
# Columns to keep
keep_columns = ['Writer_1', 'Writer_2', 'Director_1', 'Director_2']

# Determine which columns to drop
columns_to_drop = [col for col in merged_df_2.columns if ('Writer_' in col or 'Director_' in col) and col not in keep_columns]

# Drop the columns from the DataFrame
merged_df_2.drop(columns=columns_to_drop, inplace=True)

In [24]:
print(merged_df_2)

            tconst  averageRating titleType                   primaryTitle  \
8        tt0000009            5.4     movie                     Miss Jerry   
143      tt0000147            5.2     movie  The Corbett-Fitzsimmons Fight   
336      tt0000502            4.4     movie                       Bohemios   
370      tt0000574            6.0     movie    The Story of the Kelly Gang   
378      tt0000591            5.4     movie               The Prodigal Son   
...            ...            ...       ...                            ...   
1422463  tt9916362            6.4     movie                          Coven   
1422468  tt9916428            3.6     movie            The Secret of China   
1422474  tt9916538            8.6     movie            Kuambil Lagi Hatiku   
1422487  tt9916706            7.6     movie                Dankyavar Danka   
1422489  tt9916730            7.0     movie                         6 Gunn   

        startYear    Writer_1   Writer_2  Director_1 Director_2

In [25]:
# Save the dataset as CSV
merged_df_2.to_csv('merged_dataset_2.csv', index=False)